# Spanish Language Tutor
The purpose of this app is to be a spanish tutor companion to help english spoken students learn spanish.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = """
    You are a language tutor and help students whose main language is english to learn spanish. As english is the main language of the students you need provide your answers in english and use spanish as needed.
"""

In [ ]:
!pip install simpleaudio

In [ ]:
import time
from pydub import AudioSegment
from io import BytesIO
import tempfile
import os
import simpleaudio as sa

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # Also, try replacing onyx with alloy
        input=message
    )

    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")

    # Create a temporary file in a folder where you have write permissions
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False, dir=os.path.expanduser("~/Documents")) as temp_audio_file:
        temp_file_name = temp_audio_file.name
        audio.export(temp_file_name, format="wav")

    # Load and play audio using simpleaudio
    try:
        wave_obj = sa.WaveObject.from_wave_file(temp_file_name)
        print("start talking")
        wave_obj.play()
        audio_duration = len(audio) / 1000.0  # pydub duration in seconds
        time.sleep(audio_duration + 0.5)  # Add small buffer
        print("end talking")
        # Clean up the temporary file afterward
        os.remove(temp_file_name)
    except Exception as e:
        print(f"Audio device error: {e}")

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    reply = response.choices[0].message.content
    talker(reply)
    return response.choices[0].message.content



In [ ]:
chat("""how do say "I'm hungry""", [])

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()